# Enumerate Facet inequalities for a given number of inputs and binary outputs
Here I want to implement an algorithm, to find all facet inequalities for a given number of inputs and binary outputs.
This follows the construction of *Algorithm 1* in the paper [*Bell inequalities from no-signalling distribution*, *Cope & Colbeck (2019)*](https://arxiv.org/abs/1812.10017).

First we set the number of inputs and outputs and obtain the extremal points for the specific NS polytope.

In [1]:
#%load_ext snakeviz


from linearbell.utils import extremal_ns_binary_vertices, get_deterministic_behaviors, get_allowed_relabellings
import numpy as np
# set inputs / outputs
inputs_a = range(3)
inputs_b = range(3)
outputs = range(2)
epsilons = np.linspace(1/3,2/3, num=0)
file = '../data/facets/{}{}{}{}_{}eps.txt'.format(len(inputs_a), len(inputs_b), len(outputs), len(outputs), len(epsilons))
# get extremal points
extremals = extremal_ns_binary_vertices(inputs_a, inputs_b, outputs)

# get deterministic points
dets = get_deterministic_behaviors(inputs_a, inputs_b, outputs)

# get allowed relabellings
allowed_relabellings = get_allowed_relabellings(inputs_a, inputs_b, outputs, outputs)

# options for local weight optimizer (bland is only needed for higher dimensions (m_a , m_b > 4)
options = {"disp": False, "maxiter": 5000, "bland": True}
method='simplex'

In [2]:
from linearbell.utils import find_local_weight, facet_inequality_check, check_equiv_bell
from itertools import product


# list of all facets
facets = []
# iterate through extremals

for z, e in enumerate(extremals):
    print('{} / {} : num of facets: {}'.format(z,len(extremals),len(facets)))
    # find the local weight for the extremal behavior
    opt, bell_expression = find_local_weight(e, dets, method=method, options=options)

    # check that there is no local part, as it is an extremal point
    assert np.abs(bell_expression @ e) < 1e-6, 'local weight of extremal is not zero: {}'.format(np.abs(bell_expression @ e))
    # find all equalizing local behaviors for the inequality bell_expression @ d >= 1
    is_facet, bell_expression, eq_dets = facet_inequality_check(dets, bell_expression, len(inputs_a), len(inputs_b), len(outputs))
    # check if other representation is already in facets
    if is_facet:
        for i in range(len(facets)-1,-1,-1):
            if check_equiv_bell(bell_expression, facets[i], allowed_relabellings, dets):
                is_facet = False
                break
    if is_facet:
        facets.append(bell_expression)
        print('num facets: {}'.format(len(facets)))
    # iterate through epsilons
    for epsilon in epsilons:
        # define new behaviors based on epsilon
        for j in range(eq_dets.shape[0]):
            for k in range(j, eq_dets.shape[0]):
                # form new probability distribution
                e_new = (1-3*epsilon/2)*e + epsilon*eq_dets[j] + epsilon / 2 * eq_dets[k]
                # do the facet check again
                opt, bell_expression = find_local_weight(e_new, dets,method=method, options=options)

                is_facet, bell_expression, eq_dets_new = facet_inequality_check(dets, bell_expression, len(inputs_a), len(inputs_b), len(outputs))
                # do the same checks as above, could be done in a better way
                if not is_facet: continue
                for i in range(len(facets)-1,-1,-1):
                    if check_equiv_bell(bell_expression, facets[i], allowed_relabellings, dets):
                        is_facet = False
                        break
                if not is_facet: continue
                facets.append(bell_expression)
                print('num facets: {}'.format(len(facets)))
print('Number of facet classes found: {}'.format(len(facets)))
facets = np.array(facets)
np.savetxt(file, facets)

0 / 8 : num of facets: 0
num facets: 1
1 / 8 : num of facets: 1
2 / 8 : num of facets: 1
3 / 8 : num of facets: 1
4 / 8 : num of facets: 1
5 / 8 : num of facets: 1
6 / 8 : num of facets: 1
num facets: 2
7 / 8 : num of facets: 2
Number of facet classes found: 2


Now we have found the number of classes of facets for a specific number of inputs and outputs.
As this is just the number of classes, we could store all the facet inequalities belonging to that class along the way.

